In [3]:
import cv2
import mediapipe as mp

# Inicializar MediaPipe Face Detection
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

# Cargar la imagen del sombrero y convertirla a RGBA
hat = cv2.imread('hat-crop.png', cv2.IMREAD_UNCHANGED)
hat_h, hat_w, _ = hat.shape  # Obtener dimensiones del sombrero

# Cargar la imagen del collar y convertirla a RGBA
ruff = cv2.imread('ruff-crop.png', cv2.IMREAD_UNCHANGED)
ruff_h, ruff_w, _ = hat.shape  # Obtener dimensiones del collar

# Variables de estado anteriores para suavizado
prev_x, prev_y, prev_w, prev_h = None, None, None, None
alpha = 0.2  # Factor de suavizado; entre más cercano a 1, menos suavizado

# Inicializar la cámara o cargar un video
cap = cv2.VideoCapture(0)  # Usa '0' para la cámara web

# Función para dibujar un rectángulo con esquinas redondeadas
def draw_rounded_rectangle(image, top_left, bottom_right, color, corner_radius, thickness=-1):
    x1, y1 = top_left
    x2, y2 = bottom_right

    # Dibujar círculos en las esquinas
    cv2.circle(image, (x1 + corner_radius, y1 + corner_radius), corner_radius, color, thickness)
    cv2.circle(image, (x2 - corner_radius, y1 + corner_radius), corner_radius, color, thickness)
    cv2.circle(image, (x1 + corner_radius, y2 - corner_radius), corner_radius, color, thickness)
    cv2.circle(image, (x2 - corner_radius, y2 - corner_radius), corner_radius, color, thickness)

    # Dibujar líneas entre los círculos
    cv2.rectangle(image, (x1 + corner_radius, y1), (x2 - corner_radius, y2), color, thickness)
    cv2.rectangle(image, (x1, y1 + corner_radius), (x2, y2 - corner_radius), color, thickness)



with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5) as face_detection:
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            print("No se puede acceder a la cámara.")
            break

       

        # Convertir la imagen a RGB
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Procesar la imagen para detectar caras
        results = face_detection.process(image_rgb)

        # Convertir de nuevo a BGR para OpenCV
        image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

        background_layer = image_bgr.copy() 
        
        # Dibujar los resultados de la detección en la imagen
        if results.detections:
            for detection in results.detections:
                # Dibuja el cuadro de la cara detectada
                #mp_drawing.draw_detection(image_bgr, detection)

                # Extraer el bounding box de la detección
                bboxC = detection.location_data.relative_bounding_box
                ih, iw, _ = image_bgr.shape  # Dimensiones de la imagen

                # Convertir coordenadas relativas a píxeles absolutos
                x = int(bboxC.xmin * iw)
                y = int(bboxC.ymin * ih)
                w = int(bboxC.width * iw)
                h = int(bboxC.height * ih)

                # Suavizado exponencial
                if prev_x is not None:
                    x = int(alpha * x + (1 - alpha) * prev_x)
                    y = int(alpha * y + (1 - alpha) * prev_y)
                    w = int(alpha * w + (1 - alpha) * prev_w)
                    h = int(alpha * h + (1 - alpha) * prev_h)
                
                # Actualizar las variables anteriores
                prev_x, prev_y, prev_w, prev_h = x, y, w, h

                # Copia temporal de la región de la cara en la bounding box
                face_region = image_bgr[y:y+h, x:x+w].copy()

                 # Calcular el rectángulo con proporción de carta de póker (5:7)
                card_w = int(w * 1.4)
                card_h = int(card_w * 7 / 5)
                card_x = x - int((card_w - w) / 2)
                card_y = y - int(h * 0.3)

                # Limitar los bordes del rectángulo dentro de la imagen
                card_x = max(0, card_x)
                card_y = max(0, card_y)
                card_w = min(iw - card_x, card_w)
                card_h = min(ih - card_y, card_h)

                # Dibujar el rectángulo de fondo con esquinas redondeadas
                corner_radius = int(0.1 * card_w)  # Ajuste del radio de las esquinas
                draw_rounded_rectangle(image_bgr, (card_x, card_y), (card_x + card_w, card_y + card_h), (255, 255, 255), corner_radius)

                # Superponer la región de la cara en la carta
                image_bgr[y:y+h, x:x+w] = face_region
                
                # Redimensionar el sombrero y calcular posición centrada
                new_hat_w = int(w * 1.3)  # Sombrero más ancho que la cara
                new_hat_h = int(hat_h * (new_hat_w / hat_w))  # Mantener proporción
                resized_hat = cv2.resize(hat, (new_hat_w, new_hat_h), interpolation=cv2.INTER_AREA)
                
                # Calcular la posición del sombrero centrado
                x_hat = x - int((new_hat_w - w) / 2)  # Centrar el sombrero horizontalmente
                y_hat = y - new_hat_h  # Posicionar el sombrero justo encima de la cara

                # Ajustar dimensiones para evitar que el sombrero se salga de los límites de la imagen
                hat_h_end = min(ih, y_hat + new_hat_h) - y_hat
                hat_w_end = min(iw, x_hat + new_hat_w) - x_hat
                resized_hat_cropped = resized_hat[:hat_h_end, :hat_w_end]

                # Superponer el sombrero con transparencia
                if y_hat >= 0:
                    for i in range(0, 3):  # BGR canales
                        image_bgr[y_hat:y_hat + hat_h_end, x_hat:x_hat + hat_w_end, i] = \
                            image_bgr[y_hat:y_hat + hat_h_end, x_hat:x_hat + hat_w_end, i] * (1 - resized_hat_cropped[:, :, 3] / 255.0) + \
                            resized_hat_cropped[:, :, i] * (resized_hat_cropped[:, :, 3] / 255.0)
                

                #Redimensionar el collar y calcular posición centrada
                new_ruff_w = int(w * 1.5)  # Collar más ancho que la cara
                new_ruff_h = int(ruff_h * (new_ruff_w / ruff_w))  # Mantener proporción
                resized_ruff = cv2.resize(ruff, (new_ruff_w, new_ruff_h), interpolation=cv2.INTER_AREA)

                # Calcular la posición del collar centrado
                x_ruff = x - int((new_ruff_w - w) / 2)  # Centrar el collar horizontalmente
                y_ruff = y + h - int(0.2 * new_ruff_h)  # Posicionar el collar justo debajo de la cara

                # Ajustar dimensiones para evitar que el collar se salga de los límites de la imagen
                ruff_h_end = min(ih, y_ruff + new_ruff_h) - y_ruff
                ruff_w_end = min(iw, x_ruff + new_ruff_w) - x_ruff
                resized_ruff_cropped = resized_ruff[:ruff_h_end, :ruff_w_end]

                # Superponer el collar con transparencia
                if y_ruff + ruff_h_end <= ih:
                    for i in range(0, 3):  # BGR canales
                        image_bgr[y_ruff:y_ruff + ruff_h_end, x_ruff:x_ruff + ruff_w_end, i] = \
                            image_bgr[y_ruff:y_ruff + ruff_h_end, x_ruff:x_ruff + ruff_w_end, i] * (1 - resized_ruff_cropped[:, :, 3] / 255.0) + \
                            resized_ruff_cropped[:, :, i] * (resized_ruff_cropped[:, :, 3] / 255.0)

        # Mostrar la imagen en una ventana
        cv2.imshow('Face Detection', image_bgr)

        # Presionar 'q' para salir
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

# Liberar la cámara y cerrar las ventanas
cap.release()
cv2.destroyAllWindows()


In [4]:
# Índices de los landmarks que definen el contorno de la cara
face_outline_indices = [
    10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288, 397, 365, 379, 
    378, 400, 377, 152, 148, 176, 149, 150, 136, 172, 58, 132, 93, 234, 127, 
    162, 21, 54, 103, 67, 109
]

left_eye_indices = [
    33, 7, 163, 144, 145, 153, 154, 155, 133, 157, 158, 159, 160, 161, 246
]

right_eye_indices = [
    362, 398, 384, 385, 386, 387, 388, 466, 373, 374, 380, 249
]

upper_mouth_indices = [
    61, 185, 40, 39, 37, 0, 267, 270, 409, 291, 375, 321, 405, 314, 17, 84, 181
]

lower_mouth_indices = [
    146, 91, 181, 84, 17, 314, 405, 321, 375, 291, 409, 270, 267, 0, 37, 39, 40, 185
]

eyes_indices = left_eye_indices + right_eye_indices
mouth_indices = upper_mouth_indices + lower_mouth_indices

# Función para dibujar un rectángulo con esquinas redondeadas
def draw_rounded_rectangle(image, top_left, bottom_right, color, corner_radius, thickness=-1):
    x1, y1 = top_left
    x2, y2 = bottom_right

    # Dibujar círculos en las esquinas
    cv2.circle(image, (x1 + corner_radius, y1 + corner_radius), corner_radius, color, thickness)
    cv2.circle(image, (x2 - corner_radius, y1 + corner_radius), corner_radius, color, thickness)
    cv2.circle(image, (x1 + corner_radius, y2 - corner_radius), corner_radius, color, thickness)
    cv2.circle(image, (x2 - corner_radius, y2 - corner_radius), corner_radius, color, thickness)

    # Dibujar líneas entre los círculos
    cv2.rectangle(image, (x1 + corner_radius, y1), (x2 - corner_radius, y2), color, thickness)
    cv2.rectangle(image, (x1, y1 + corner_radius), (x2, y2 - corner_radius), color, thickness)

In [12]:
import cv2
import mediapipe as mp
import numpy as np
import math

# Inicializar MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5)

# Inicializar la cámara
cap = cv2.VideoCapture(0)


try:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("No se puede acceder a la cámara.")
            break

        # Convertir la imagen a RGB para procesarla con MediaPipe
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Procesar la imagen para detectar landmarks faciales
        results = face_mesh.process(image_rgb)
        
        # Convertir la imagen de nuevo a BGR para OpenCV
        image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

        # Verificar si se detectaron caras
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                # Obtener los puntos específicos que forman el contorno de la cara
                ih, iw, _ = image_bgr.shape
                for i in face_outline_indices:
                    x = int(face_landmarks.landmark[i].x * iw)
                    y = int(face_landmarks.landmark[i].y * ih)
                    cv2.circle(image_bgr, (x, y), 2, (0, 255, 0), -1)
                    # Poner el número del landmark
                    cv2.putText(image_bgr, str(i), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1, cv2.LINE_AA)

                contour_points = [
                    (int(face_landmarks.landmark[i].x * iw), int(face_landmarks.landmark[i].y * ih)) 
                    for i in face_outline_indices
                ]

                cv2.polylines(image_bgr, [np.array(contour_points)], isClosed=True, color=(255, 255, 255), thickness=2)

                # Crear una máscara para los ojos
                eye_mask = np.zeros((ih, iw), dtype=np.uint8)
                
                # Obtener los puntos de los ojos y dibujar contornos en la máscara
                left_eye_points = [(int(face_landmarks.landmark[i].x * iw), int(face_landmarks.landmark[i].y * ih)) for i in left_eye_indices]
                right_eye_points = [(int(face_landmarks.landmark[i].x * iw), int(face_landmarks.landmark[i].y * ih)) for i in right_eye_indices]

                # Dibujar el contorno de los ojos en la máscara (grosor pequeño inicial)
                cv2.polylines(eye_mask, [np.array(left_eye_points)], isClosed=True, color=255, thickness=1)
                cv2.polylines(eye_mask, [np.array(right_eye_points)], isClosed=True, color=255, thickness=1)

                # Dilatar la máscara para hacer que el contorno se expanda hacia afuera
                dilated_eye_mask = cv2.dilate(eye_mask, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)))

                # Crear un overlay en color rojo donde esté la máscara dilatada
                overlay_eyes = np.zeros_like(image_bgr)
                overlay_eyes[dilated_eye_mask == 255] = (0, 0, 255)  # Rojo en BGR

                # Aplicar la transparencia para que el contorno no sea demasiado fuerte
                alpha = 0.4  # Nivel de transparencia
                image_bgr = cv2.addWeighted(overlay_eyes, alpha, image_bgr, 1 - alpha, 0)


                # Crear una máscara para los ojos
                mouth_mask = np.zeros((ih, iw), dtype=np.uint8)
                
                # Obtener los puntos de los ojos y dibujar contornos en la máscara
                upper_mouth_points = [(int(face_landmarks.landmark[i].x * iw), int(face_landmarks.landmark[i].y * ih)) for i in upper_mouth_indices]
                lower_mouth_points = [(int(face_landmarks.landmark[i].x * iw), int(face_landmarks.landmark[i].y * ih)) for i in lower_mouth_indices]

                # Dibujar el contorno de los ojos en la máscara (grosor pequeño inicial)
                cv2.polylines(mouth_mask, [np.array(left_eye_points)], isClosed=True, color=255, thickness=1)
                cv2.polylines(mouth_mask, [np.array(right_eye_points)], isClosed=True, color=255, thickness=1)

                # Dilatar la máscara para hacer que el contorno se expanda hacia afuera
                dilated_mouth_mask = cv2.dilate(mouth_mask, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)))

                # Crear un overlay en color rojo donde esté la máscara dilatada
                overlay_mouth = np.zeros_like(image_bgr)
                overlay_mouth[dilated_mouth_mask == 255] = (0, 0, 255)  # Rojo en BGR

                # Aplicar la transparencia para que el contorno no sea demasiado fuerte
                alpha = 0.4  # Nivel de transparencia
                image_bgr = cv2.addWeighted(overlay_mouth, alpha, image_bgr, 1 - alpha, 0)

                



        # Mostrar el resultado
        cv2.imshow('Face Contour', image_bgr)

        # Salir con la tecla 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    cap.release()
    cv2.destroyAllWindows()


In [5]:
import cv2
import mediapipe as mp
import numpy as np
import math

# Inicializar MediaPipe Face Mesh (para obtener los landmarks)
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# Cargar la imagen del sombrero y collar (sin cambios)
hat = cv2.imread('hat-crop.png', cv2.IMREAD_UNCHANGED)
hat_h, hat_w, _ = hat.shape

ruff = cv2.imread('ruff-crop.png', cv2.IMREAD_UNCHANGED)
ruff_h, ruff_w, _ = ruff.shape

# Inicializar la cámara
cap = cv2.VideoCapture(0)


# Inicializar Face Mesh
with mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("No se puede acceder a la cámara.")
            break

        # Convertir la imagen a RGB
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Procesar la imagen para obtener los landmarks de la cara
        results = face_mesh.process(image_rgb)

        # Convertir de nuevo a BGR para OpenCV
        image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                ih, iw, _ = image_bgr.shape
                
                # Obtener los puntos del contorno de la cara
                contour_points = [
                    (int(face_landmarks.landmark[i].x * iw), int(face_landmarks.landmark[i].y * ih)) for i in face_outline_indices
                ]

                x, y, w, h = cv2.boundingRect(np.array(contour_points))
                
                # Obtener rotación de la cara a partir de la línea superior de la cara (54,284)
                ul_x, ul_y = int(face_landmarks.landmark[54].x * iw), int(face_landmarks.landmark[54].y * ih)    # Esquina superior izquierda
                ur_x, ur_y = int(face_landmarks.landmark[284].x * iw), int(face_landmarks.landmark[284].y * ih)  # Esquina superior derecha

                # Calcular la diferencia en x y en y
                dx = ur_x - ul_x
                dy = ur_y - ul_y

                # Calcular el ángulo de rotación
                angle_deg = np.degrees(np.arctan2(dy, dx))
                
                # Calcular la posición base del sombrero
                # Tomamos el punto medio de los puntos superior izquierdo y derecho de la cara
                center_x = (ul_x + ur_x) // 2
                center_y = (ul_y + ur_y) // 2

                # Hipotenusa del triańgulo rectángulo formado la diferencia en x y en y
                hypotenuse = math.sqrt(dy ** 2 + dx ** 2)

                # Redimensionar el sombrero y calcular posición centrada
                new_hat_w = int(hypotenuse)*2  # Sombrero más ancho que la cara
                new_hat_h = int(hat_h * (new_hat_w / hat_w))  # Mantener proporción
                resized_hat = cv2.resize(hat, (new_hat_w, new_hat_h), interpolation=cv2.INTER_AREA)

                # Crear la matriz de rotación centrada en el centro del sombrero
                center_hat = (new_hat_w // 2, new_hat_h // 2)
                rotation_matrix = cv2.getRotationMatrix2D(center_hat, -angle_deg, 1.0)

                # Aplicar la rotación al sombrero
                rotated_hat = cv2.warpAffine(resized_hat, rotation_matrix, (new_hat_w, new_hat_h), 
                                            flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REPLICATE)
                
                rotated_hat_h, rotated_hat_w, _ = rotated_hat.shape

                # Posición inicial del sombrero en la imagen, centrado con el punto medio de los ojos
                x_hat = center_x - rotated_hat.shape[1] // 2 
                y_hat = center_y - (rotated_hat.shape[0] // 2) - int(0.3 * h)  # Posicionar el sombrero justo encima de la cara

                
                # Ajustar dimensiones para evitar que el sombrero se salga de los límites de la imagen
                hat_h_end = min(ih, y_hat + rotated_hat.shape[0]) - y_hat
                hat_w_end = min(iw, x_hat + rotated_hat.shape[1]) - x_hat
                rotated_hat_cropped = rotated_hat[:hat_h_end, :hat_w_end]

                # Superponer el sombrero rotado sobre la imagen
                if y_hat >= 0:
                    for i in range(0, 3):  # Para cada canal de color (BGR)
                        image_bgr[y_hat:y_hat + hat_h_end, x_hat:x_hat + hat_w_end, i] = \
                            image_bgr[y_hat:y_hat + hat_h_end, x_hat:x_hat + hat_w_end, i] * (1 - rotated_hat_cropped[:, :, 3] / 255.0) + \
                            rotated_hat_cropped[:, :, i] * (rotated_hat_cropped[:, :, 3] / 255.0)


                #Redimensionar el collar y calcular posición centrada
                new_ruff_w = int(w * 1.5)  # Collar más ancho que la cara
                new_ruff_h = int(ruff_h * (new_ruff_w / ruff_w))  # Mantener proporción
                resized_ruff = cv2.resize(ruff, (new_ruff_w, new_ruff_h), interpolation=cv2.INTER_AREA)

                # Calcular la posición del collar centrado
                x_ruff = x - int((new_ruff_w - w) / 2)  # Centrar el collar horizontalmente
                y_ruff = y + h - int(0.2 * new_ruff_h)  # Posicionar el collar justo debajo de la cara

                # Ajustar dimensiones para evitar que el collar se salga de los límites de la imagen
                ruff_h_end = min(ih, y_ruff + new_ruff_h) - y_ruff
                ruff_w_end = min(iw, x_ruff + new_ruff_w) - x_ruff
                resized_ruff_cropped = resized_ruff[:ruff_h_end, :ruff_w_end]

                # Superponer el collar con transparencia
                if y_ruff + ruff_h_end <= ih:
                    for i in range(0, 3):  # BGR canales
                        image_bgr[y_ruff:y_ruff + ruff_h_end, x_ruff:x_ruff + ruff_w_end, i] = \
                            image_bgr[y_ruff:y_ruff + ruff_h_end, x_ruff:x_ruff + ruff_w_end, i] * (1 - resized_ruff_cropped[:, :, 3] / 255.0) + \
                            resized_ruff_cropped[:, :, i] * (resized_ruff_cropped[:, :, 3] / 255.0)
                        
                cv2.circle(image_bgr, (ul_x, ul_y), 2, (0, 255, 0), -1)
                cv2.circle(image_bgr, (ur_x, ur_y), 2, (0, 255, 0), -1)
                cv2.circle(image_bgr, (center_x, center_y), 2, (0, 255, 0), -1)

                cv2.circle(image_bgr, (x_hat, y_hat + rotated_hat_h), 2, (255, 0, 0), -1)
                cv2.circle(image_bgr, (x_hat + rotated_hat_w, y_hat + rotated_hat_h), 2, (255, 0, 0), -1)
                cv2.circle(image_bgr, (x_hat + rotated_hat_w // 2, y_hat + rotated_hat_h // 2), 2, (255, 0, 0), -1)


        # Mostrar la imagen con el filtro aplicado
        cv2.imshow('Face Filter', image_bgr)

        # Salir si se presiona la tecla 'q'
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

# Liberar la cámara y cerrar las ventanas
cap.release()
cv2.destroyAllWindows()


                # Crear una máscara para la cara (un área cerrada con los landmarks)
                mask = np.zeros((ih, iw), dtype=np.uint8)
                cv2.fillPoly(mask, [np.array(contour_points)], 255)

                # Crear una región de la cara usando la máscara
                face_region = cv2.bitwise_and(image_bgr, image_bgr, mask=mask)

                # Redibujar la carta
                # Calcular el tamaño de la carta basado en la cara detectada
                x, y, w, h = cv2.boundingRect(np.array(contour_points))  # Bounding box ajustada al contorno de la cara
                card_w = int(w * 1.4)
                card_h = int(card_w * 7 / 5)
                card_x = max(0, x - int((card_w - w) / 2))
                card_y = max(0, y - int(h * 0.3))

                # Limitar las dimensiones dentro de la imagen
                card_w = min(iw - card_x, card_w)
                card_h = min(ih - card_y, card_h)

                # Redibujar la carta (puedes añadir un fondo de color o transparencia aquí)
                corner_radius = int(0.1 * card_w)
                draw_rounded_rectangle(image_bgr, (card_x, card_y), (card_x + card_w, card_y + card_h), (255, 255, 255), corner_radius)





                # Calcular la diferencia de coordenadas del centro respecto al frame anterior
                if prev_x is not None:
                    dx_hat = face_center_x - prev_x
                    dy_hat = face_center_y - prev_y

                # Posición inicial del sombrero en la imagen
                x_hat = prev_x + dx_hat if prev_x is not None else face_center_x - rotated_hat_w // 2
                y_hat = prev_y + dy_hat if prev_y is not None else face_center_y - rotated_hat_h 

                # Actualizar las variables anteriores
                prev_x, prev_y = face_center_x, face_center_y



                                # Ajustar dimensiones para evitar que el sombrero se salga de los límites de la imagen
                hat_h_end = min(ih, y_hat + rotated_hat.shape[0]) - y_hat
                hat_w_end = min(iw, x_hat + rotated_hat.shape[1]) - x_hat
                x_hat = max(0, x_hat)
                y_hat = max(0, y_hat)
                rotated_hat_cropped = rotated_hat[:hat_h_end, :hat_w_end]


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import math

# Inicializar MediaPipe Face Mesh (para obtener los landmarks)
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# Cargar la imagen del sombrero y collar (sin cambios)
hat = cv2.imread('hat-crop.png', cv2.IMREAD_UNCHANGED)
hat_h, hat_w, _ = hat.shape

ruff = cv2.imread('ruff-crop.png', cv2.IMREAD_UNCHANGED)
ruff_h, ruff_w, _ = ruff.shape

# Inicializar la cámara
cap = cv2.VideoCapture(0)


# Inicializar Face Mesh
with mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("No se puede acceder a la cámara.")
            break

        # Convertir la imagen a RGB
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Procesar la imagen para obtener los landmarks de la cara
        results = face_mesh.process(image_rgb)

        # Convertir de nuevo a BGR para OpenCV
        image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                ih, iw, _ = image_bgr.shape
                
                # Obtener los puntos del contorno de la cara
                contour_points = [
                    (int(face_landmarks.landmark[i].x * iw), int(face_landmarks.landmark[i].y * ih)) for i in face_outline_indices
                ]

                x, y, w, h = cv2.boundingRect(np.array(contour_points))
                

                # LÍNEA SUPERIOR DE LA CARA
                # Obtener rotación de la cara a partir de la línea superior de la cara (54,284)
                ul_x, ul_y = int(face_landmarks.landmark[54].x * iw), int(face_landmarks.landmark[54].y * ih)    # Esquina superior izquierda
                ur_x, ur_y = int(face_landmarks.landmark[284].x * iw), int(face_landmarks.landmark[284].y * ih)  # Esquina superior derecha

                # CENTRO DE LA LÍNEA SUPERIOR DE LA CARA
                face_center_x = (ul_x + ur_x) // 2
                face_center_y = (ul_y + ur_y) // 2

                # Calcular la diferencia en x y en y
                dx = ur_x - ul_x
                dy = ur_y - ul_y

                # LONGITUD DE LA LÍNEA SUPERIOR DE LA CARA
                face_line_length = math.sqrt(dy ** 2 + dx ** 2)

                # Calcular el ángulo de rotación
                angle_deg = np.degrees(np.arctan2(dy, dx)) 


                # Redimensionar el sombrero y calcular posición centrada
                new_hat_w = int(face_line_length)*2  # Sombrero más ancho que la cara
                new_hat_h = int(hat_h * (new_hat_w / hat_w))  # Mantener proporción
                resized_hat = cv2.resize(hat, (new_hat_w, new_hat_h), interpolation=cv2.INTER_AREA)

                # Crear la matriz de rotación centrada en el centro del sombrero
                hat_center = (new_hat_w // 2, new_hat_h // 2)
                rotation_matrix = cv2.getRotationMatrix2D(hat_center, -angle_deg, 1.0)

                # Aplicar la rotación al sombrero
                rotated_hat = cv2.warpAffine(resized_hat, rotation_matrix, (new_hat_w, new_hat_h), 
                                            flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REPLICATE)
                
                rotated_hat_h, rotated_hat_w, _ = rotated_hat.shape

                # Posición inicial del sombrero en la imagen, centrado con el punto medio de los ojos
                x_hat = face_center_x - rotated_hat_w // 2 
                y_hat = face_center_y - (rotated_hat_h // 2) - int(0.3 * h)  # Posicionar el sombrero justo encima de la cara

                hat_center_x = new_hat_w // 2 + x_hat
                hat_center_y = new_hat_h // 2 + y_hat

                angle = -angle_deg
                beta = 90 - angle
                h = hat_center_y - face_center_y
                ab = h / math.sin(math.radians(beta))
                bc = int(ab / math.cos(math.radians(beta)))

                if 0 <= angle < 90:
                    x_hat -= bc
                if -90 < angle < 0:
                    x_hat += bc

                # Ajustar dimensiones para evitar que el sombrero se salga de los límites de la imagen
                hat_h_end = min(ih, y_hat + rotated_hat_h) - y_hat
                hat_w_end = min(iw, x_hat + rotated_hat_w) - x_hat
                rotated_hat_cropped = rotated_hat[:hat_h_end, :hat_w_end]

                # Superponer el sombrero rotado sobre la imagen
                if y_hat >= 0:
                    for i in range(0, 3):  # Para cada canal de color (BGR)
                        fondo = image_bgr[y_hat:y_hat + hat_h_end, x_hat:x_hat + hat_w_end, i] * (1 - rotated_hat_cropped[:, :, 3] / 255.0)
                        print("FONDO", fondo.shape)
                        imagen = rotated_hat_cropped[:, :, i] * (rotated_hat_cropped[:, :, 3] / 255.0)
                        print("IMAGEN", imagen.shape)
                        image_bgr[y_hat:y_hat + hat_h_end, x_hat:x_hat + hat_w_end, i] = fondo + imagen


                #Redimensionar el collar y calcular posición centrada
                new_ruff_w = int(w * 1.5)  # Collar más ancho que la cara
                new_ruff_h = int(ruff_h * (new_ruff_w / ruff_w))  # Mantener proporción
                resized_ruff = cv2.resize(ruff, (new_ruff_w, new_ruff_h), interpolation=cv2.INTER_AREA)

                # Calcular la posición del collar centrado
                x_ruff = x - int((new_ruff_w - w) / 2)  # Centrar el collar horizontalmente
                y_ruff = y + h - int(0.2 * new_ruff_h)  # Posicionar el collar justo debajo de la cara

                # Ajustar dimensiones para evitar que el collar se salga de los límites de la imagen
                ruff_h_end = min(ih, y_ruff + new_ruff_h) - y_ruff
                ruff_w_end = min(iw, x_ruff + new_ruff_w) - x_ruff
                resized_ruff_cropped = resized_ruff[:ruff_h_end, :ruff_w_end]

                # Superponer el collar con transparencia
                if y_ruff + ruff_h_end <= ih:
                    for i in range(0, 3):  # BGR canales
                        fondo = image_bgr[y_ruff:y_ruff + ruff_h_end, x_ruff:x_ruff + ruff_w_end, i] * (1 - resized_ruff_cropped[:, :, 3] / 255.0)
                        imagen = resized_ruff_cropped[:, :, i] * (resized_ruff_cropped[:, :, 3] / 255.0)
                        image_bgr[y_ruff:y_ruff + ruff_h_end, x_ruff:x_ruff + ruff_w_end, i] = fondo + imagen
                            
                        
                cv2.circle(image_bgr, (ul_x, ul_y), 2, (0, 255, 0), -1)
                cv2.circle(image_bgr, (ur_x, ur_y), 2, (0, 255, 0), -1)
                cv2.circle(image_bgr, (face_center_x, face_center_y), 2, (0, 255, 0), -1)

                #cv2.circle(image_bgr, (x_hat, y_hat + rotated_hat_h), 2, (255, 0, 0), -1)
                #cv2.circle(image_bgr, (x_hat + rotated_hat_w, y_hat + rotated_hat_h), 2, (255, 0, 0), -1)
                #cv2.circle(image_bgr, (x_hat + rotated_hat_w // 2, y_hat + rotated_hat_h // 2), 2, (255, 0, 0), -1)


        # Mostrar la imagen con el filtro aplicado
        cv2.imshow('Face Filter', image_bgr)

        # Salir si se presiona la tecla 'q'
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

# Liberar la cámara y cerrar las ventanas
cap.release()
cv2.destroyAllWindows()
